In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import statsmodels.api as sm
import plotly.graph_objects as go

In [20]:
# Load sugarscape model csv data and ignore the first 6 rows
df = pd.read_csv(os.path.join('data', '2 Sugarscape 3 Wealth Distribution meta vs growth-table.csv'), skiprows=6)

# Remove any rows where columns "max-vis >= min-vis" and "max-meta >= min-meta" are the string "False"
df = df[(df['max-meta >= min-meta'] == True)]

# Remove unnecessary columns
df = df.drop(columns=['max-vis >= min-vis', 'max-meta >= min-meta', 'visualization',
                       '[run number]', 'initial-population', '[step]', 'max-vis', 'min-vis'], errors='ignore')

# Rename columns
df = df.rename(columns={'mean [metabolism] of turtles:': 'mean_metabolism',
                        'mean [vision] of turtles:': 'mean_vision',
                        'count turtles': 'turtles_count',
                        '(gini-index-reserve / count turtles) * 2': 'gini_index'})

# Convert all columns to numeric
df = df.apply(pd.to_numeric, errors='coerce')

# Remove rows with NaN
df = df.dropna()

In [40]:
# Make a boxplot of the gini_index for each sugar-growback-rate
fig = px.box(df[df['max-meta'] == 6], x='sugar-growback-rate', y='gini_index', title='Gini Index vs Sugar Growback Rate')
fig.update_layout(xaxis_title='Sugar Growback Rate', yaxis_title='Gini Index')
fig.show()

In [21]:
# Create a new df where the rows are grouped by max-meta, min-meta, and sugar-growth-rate
grouped_df = df.groupby(['max-meta', 'min-meta', 'sugar-growback-rate']).mean().reset_index()

In [22]:
# Add a new column (range of metabolism) to the grouped df
grouped_df['range_of_meta'] = grouped_df['max-meta'] - grouped_df['min-meta']

In [23]:
grouped_df

,max-meta,min-meta,sugar-growback-rate,turtles_count,mean [metabolism] of turtles,mean [vision] of turtles,gini_index,range_of_meta
0,1,1,0.1,500.0,1.00000,3.62764,0.385985,0
1,1,1,0.2,500.0,1.00000,3.77836,0.335008,0
2,1,1,0.3,500.0,1.00000,3.59172,0.225680,0
3,1,1,0.4,500.0,1.00000,3.57256,0.228179,0
4,1,1,0.5,500.0,1.00000,3.58122,0.252605,0
...,...,...,...,...,...,...,...,...
195,10,1,1.6,500.0,2.47998,3.73630,0.514927,9
196,10,1,1.7,500.0,2.47592,3.74224,0.512324,9
197,10,1,1.8,500.0,2.48010,3.72768,0.512081,9
198,10,1,1.9,500.0,2.50182,3.72394,0.511356,9


In [24]:
from scipy.interpolate import griddata

# Assuming grouped_df contains your data

# Define grid
x = grouped_df['sugar-growback-rate']
y = grouped_df['range_of_meta']
z = grouped_df['gini_index']
xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)
xi, yi = np.meshgrid(xi, yi)

# Interpolate
zi = griddata((x, y), z, (xi, yi), method='linear')

# Create surface plot
fig = go.Figure(data=[go.Surface(x=xi, y=yi, z=zi)])

# Update layout
fig.update_layout(scene=dict(
                    xaxis_title='Sugar Growback Rate',
                    yaxis_title='Range of Meta',
                    zaxis_title='Gini Index'))

fig.show()

In [25]:
# Make another plot like the one above, but instead of points created a surface plot
fig = px.scatter_3d(grouped_df, x='sugar-growback-rate', y='range_of_meta', z='gini_index')
fig.update_traces(mode='markers')
fig.add_traces(px.scatter_3d(grouped_df, x='sugar-growback-rate', y='range_of_meta', z='gini_index').data)
fig.update_traces(marker=dict(size=5))
fig.update_traces(mode='markers', marker=dict(line=dict(width=2, color='DarkSlateGrey')))
fig.update_layout(scene = dict(
                    xaxis_title='sugar-growback-rate',
                    yaxis_title='range_of_meta',
                    zaxis_title='gini_index'))
fig.show()

In [26]:
# Do a linear regression on the data using statsmodels
X = grouped_df[['sugar-growback-rate', 'range_of_meta']]
y = grouped_df['gini_index']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             gini_index   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                     155.3
Date:                Mon, 26 Feb 2024   Prob (F-statistic):           3.23e-41
Time:                        16:08:21   Log-Likelihood:                 345.16
No. Observations:                 200   AIC:                            -684.3
Df Residuals:                     197   BIC:                            -674.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.3031      0.008     37.952      0.000       0.287       0.319
sugar-growback-rate     0.0642      0.005     12.058      0.000       0.054       0.075
range_of_meta           0.0137      0.001     12.855      0.000       0.012       0.016
==============================================================================
Omnibus:                        8.446   Durbin-Watson:                   0.414
Prob(Omnibus):                  0.015   Jarque-Bera (JB):                8.536
Skew:                          -0.471   Prob(JB):                       0.0140
Kurtosis:                       2.633   Cond. No.                         16.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""